## Calculate Phi spatial average within a basin or within LMEs

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
tres = 'monthly'
#tres = 'yearly'
a1,a2,b1,b2 = [240,380,125,320]

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
darea = darea*dz/dz
darea = darea.transpose("z_t","nlat","nlon")

In [5]:
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            darea[kk:, xx, yy] = float("nan")

In [6]:
filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_1948-2017_{tres}.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, time: 840)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables: (12/13)
    Phi_0    (time, z_t, nlat, nlon) float32 ...
    Phi_1    (time, z_t, nlat, nlon) float32 ...
    Phi_2    (time, z_t, nlat, nlon) float32 ...
    Phi_3    (time, z_t, nlat, nlon) float32 ...
    Phi_4    (time, z_t, nlat, nlon) float32 ...
    Phi_5    (time, z_t, nlat, nlon) float32 ...
    ...       ...
    Phi_7    (time, z_t, nlat, nlon) float32 ...
    Phi_8    (time, z_t, nlat, nlon) float32 ...
    Phi_9    (time, z_t, nlat, nlon) float32 ...
    Phi_10   (time, z_t, nlat, nlon) float32 ...
    Phi_11   (time, z_t, nlat, nlon) float32 ...
    Phi_12   (time, z_t, nlat, nlon) float32 ...

In [7]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
lme = [1,2,3,4,5,6,7,8,9,10,65]

In [8]:
if tres == 'yearly':
    tnam = 'year'    
else:
    tnam = 'time'
nmon = ds.dims[tnam]
nver = ds.dims['z_t']

In [9]:
dsf = xr.Dataset()
for ii in range(13):
    dsf[f'Phi_{ii}'] = ds[f'Phi_{ii}']*darea

In [10]:
phi_box = np.ma.zeros([13, len(lme), nmon, nver])
phi_box.mask = True
# select a LME
for box,nn in zip(lme,range(len(lme))):
    print(box)
    latloc, lonloc = np.where(lme_mask == box)
    ar1 = np.ma.zeros([len(latloc),nver]);ar1.mask=True
    ar2 = np.ma.zeros([13, len(latloc),nmon,nver]);ar2.mask=True
    for bb in range(len(latloc)):
        ar1[bb] = darea.isel(nlat=latloc[bb], nlon=lonloc[bb])
        for ii in range(13):
            ar2[ii,bb] = dsf[f'Phi_{ii}'].isel(nlat=latloc[bb], nlon=lonloc[bb])
    #
    tarea = np.nansum(ar1,axis=0)
    tarea = np.tile(tarea[np.newaxis,np.newaxis,:],[13,nmon,1])
    phi_box[:,nn] = np.nansum(ar2,axis=1)/tarea   

1
2
3
4
5
6
7
8
9
10
65


In [11]:
dsf = xr.Dataset()
for ii in range(13):
    dsf[f'Phi_{ii}'] = xr.DataArray(data=phi_box[ii], coords=dict(lme=('lme', lme), time=('time', ds.coords[tnam].values), z_t=('z_t', ds.coords['z_t'].values)), dims=('lme','time','z_t'))

In [12]:
dsf.load()
dsf.to_netcdf(f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_horizontal_mean_LME_1948-2017_{tres}.nc', mode='w')